# Uniview module for Blazar TXS 0506+056, for IceCube-170922A

*Aaron Geller, 2018*

### Imports and function definitions

In [55]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data"


import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir,  os.pardir, os.pardir) )))
#print(sys.path)
import uvmodlib.v1 as uvmod

### USES Conf Templates

In [56]:
BlazarTemplate = """mesh
{  
    data  blazar ./modules/$folderName/BlazarSolidModel_v2.2.obj
    
#    cullRadius $cr
    glslVersion 330
    enableTypeCheck true
    
    propertyCollection 
    {        
        __objectName__
        {           
            vec1f cloudAlpha 0.4 #| public | desc "alpha for clouds" | widget slider | range 0 1
            vec1f jetAlpha 1.0 #| public | desc "alpha for jet" | widget slider | range 0 1
        }
    }

    pass 
    {
        
        useDataObject blazar
        
        shader
        {
            
            type defaultMeshShader
            {
                mtrl jetMaterial
                vertexShader   ./modules/$folderName/blazar.vs
                geometryShader ./modules/$folderName/jet.gs
                fragmentShader ./modules/$folderName/jet.fs
                
                stateManagerVar __objectName__.jetAlpha  jetAlpha

                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE true
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
                
            }            
            type defaultMeshShader
            {
                mtrl blackHoleMaterial
                vertexShader   ./modules/$folderName/blazar.vs
                geometryShader ./modules/$folderName/blazar.gs
                fragmentShader ./modules/$folderName/blazar.fs
            }

            type defaultMeshShader
            {
                mtrl accretionDiskMaterial
                vertexShader   ./modules/$folderName/blazar.vs
                geometryShader ./modules/$folderName/blazar.gs
                fragmentShader ./modules/$folderName/accretionDisk.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                }  
            }

            type defaultMeshShader
            {
                mtrl exteriorDiskMaterial
                vertexShader   ./modules/$folderName/exteriorDisk.vs
                geometryShader ./modules/$folderName/blazar.gs
                fragmentShader ./modules/$folderName/exteriorDisk.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                } 
            }

            type defaultMeshShader
            {
                mtrl cloudMaterial
                vertexShader   ./modules/$folderName/exteriorDisk.vs
                geometryShader ./modules/$folderName/jet.gs
                fragmentShader ./modules/$folderName/clouds.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                } 
                
                stateManagerVar __objectName__.cloudAlpha  cloudAlpha
                
                glState
                {
                    UV_CULL_FACE_ENABLE true
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE true
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }


        }
            
    }
    

    
    
}"""

### Blazar Class

In [57]:
class BlazarClass():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("BlazarSolidModel_v2.2.obj",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("blazar.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("blazar.gs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("blazar.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("jet.gs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("jet.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("accretionDisk.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("exteriorDisk.vs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("exteriorDisk.fs",dataFolder)        
        uvmod.Utility.ensurerelativepathexsists("clouds.fs",dataFolder)        
        self.cr = 1000
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(BlazarTemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir, cr = self.cr))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [58]:
model = BlazarClass(uvmod.OrbitalObject())
scene = uvmod.Scene()
parentScene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()
generator = uvmod.Generator()

### Specify Settings and generate the module

In [59]:
scene.setname("Blazar")
scene.setparent("MilkyWay")
scene.setunit(1000.0)
scene.setentrydist(100000.)
scene.setstaticposition(391330246, 1744728860, 178257822)

model.object.setname("Blazar")
model.object.setcoord(scene.name)
model.object.setcameraradius(10)
model.object.settargetradius(2000)
model.object.setguiname("/KavliLecture/Larson/Blazar")
model.object.setscalefactor(660.0)
model.object.showatstartup(False)

#model.cr = 10000

modinfo.setname("Blazar Model")
modinfo.setauthor("Aaron Geller<sup>1</sup> and Shane Larson<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Northwestern University")
modinfo.cleardependencies()
modinfo.setdesc("A 3D model of the Blazar TXS 0506+056 to accompany the IceCube discovery.")
modinfo.setversion("1.0")

generator.generate("Blazar",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload;")

Unable to connect to Uniview


## To reload

In [47]:
from config import Settings

In [61]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload;")

### Create colormap texture

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig("data/cmap.png", bbox_inches='tight',pad_inches=0)

plot_cmap('Blues')

plt.show()

## Blazar position 

*From the [wikipedia page](https://en.wikipedia.org/wiki/TXS_0506%2B056)*

In [14]:
from astropy.coordinates import SkyCoord, Distance
from astropy import units

In [23]:
RA = "05h 09m 25.9645434784s" #right ascension
Dec= "+05d 41m 35.333636817s" #declination
z = 0.3365 #reshift

dist = Distance(z=z).to(units.pc)
coord = SkyCoord(RA, Dec, dist)

print(coord.cartesian)

(3.91330246e+08, 1.74472886e+09, 1.78257822e+08) pc
